# Travelling Salesman Problem

The Traveliing Salesman Problem (TSP) is an Vehicle Routing problem that tries to find the sortest path to visit every location yielding the lowest cost (distance, time, etc.) There are several assumptions:
* There is only one route (or one vehicle), and no capacity constraints, it means that there is no constraint to the number of locations to be visited.
* Depot is not needed to be specified, as the route is indiffernet to this feature
* Cost from visting location j from location i may or may not be symmetric
* There are no constraints on time or sequences.

In [2]:
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

## 1.Create distance callback: Our input data is a matrix where each point M[i,j] is the cost (Time or distance) to travel from node i to j.

Matrix is assumed to be symmetric, but it may not be case

In [3]:
# Distance callback
class CreateDistanceCallback(object):
    """Create callback to calculate distances between points."""
    def __init__(self):
        """Array of distances between points."""

        self.matrix = [
        [   0, 2451,  713, 1018, 1631, 1374, 2408,  213, 2571,  875, 1420, 2145, 1972], # New York
        [2451,    0, 1745, 1524,  831, 1240,  959, 2596,  403, 1589, 1374,  357,  579], # Los Angeles
        [ 713, 1745,    0,  355,  920,  803, 1737,  851, 1858,  262,  940, 1453, 1260], # Chicago
        [1018, 1524,  355,    0,  700,  862, 1395, 1123, 1584,  466, 1056, 1280,  987], # Minneapolis
        [1631,  831,  920,  700,    0,  663, 1021, 1769,  949,  796,  879,  586,  371], # Denver
        [1374, 1240,  803,  862,  663,    0, 1681, 1551, 1765,  547,  225,  887,  999], # Dallas
        [2408,  959, 1737, 1395, 1021, 1681,    0, 2493,  678, 1724, 1891, 1114,  701], # Seattle
        [ 213, 2596,  851, 1123, 1769, 1551, 2493,    0, 2699, 1038, 1605, 2300, 2099], # Boston
        [2571,  403, 1858, 1584,  949, 1765,  678, 2699,    0, 1744, 1645,  653,  600], # San Francisco
        [ 875, 1589,  262,  466,  796,  547, 1724, 1038, 1744,    0,  679, 1272, 1162], # St. Louis
        [1420, 1374,  940, 1056,  879,  225, 1891, 1605, 1645,  679,    0, 1017, 1200], # Houston
        [2145,  357, 1453, 1280,  586,  887, 1114, 2300,  653, 1272, 1017,    0,  504], # Phoenix
        [1972,  579, 1260,  987,  371,  999,  701, 2099,  600, 1162,  1200,  504,   0]] # Salt Lake City

    def Distance(self, from_node, to_node):
        #The callback is the function Distance, which takes a pair of cities (from_node and to_node)
        #and returns the corresponding entry of the distance array.
        
        return int(self.matrix[from_node][to_node])
    
    #Note: Since the routing solver does all computations with integers, 
    #any function that creates a distance callback should convert the value returned by the distance callback to an integer. 
    #In this example, the conversion is done by the Python int() function in the line

In [4]:
dist_between_nodes = CreateDistanceCallback()
type(dist_between_nodes)

__main__.CreateDistanceCallback

In [6]:
dist_callback = dist_between_nodes.Distance
type(dist_callback)

method

## 2. Create Routing Problem

In [12]:
def main():

    # Cities: Subset of locations to be visited from the input Matrix
    city_names = ["New York", "Los Angeles", "Chicago", "Minneapolis"]
    tsp_size = len(city_names)
    # The number of routes, which is 1 in the TSP.
    num_routes = 1    
    # Nodes are indexed from 0 to tsp_size - 1. The depot is the starting node of the route.
    depot = 0

    # Create routing model
    if tsp_size > 0:
        routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
        #search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        #search_parameters = search_parameters = pywrapcp.RoutingModel.DefaultModelParameters()
        search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
        
        
        # Create the distance callback, which takes two arguments (the from and to node indices)
        # and returns the distance between these nodes.
        #The solver section of the program sets the distance callback to be the Distance method in the CreateDistanceCallback class.
        dist_between_nodes = CreateDistanceCallback()
        dist_callback = dist_between_nodes.Distance
        
        #Finally, the program passes the callback to the solver:
        routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
        
        # Solve, returns a solution if any.
        assignment = routing.SolveWithParameters(search_parameters)
        if assignment:
            # Solution cost.
            print ("Total distance: " + str(assignment.ObjectiveValue()) + " miles\n")
            # Inspect solution.

            route_number = 0
            index = routing.Start(route_number) # Index of the variable for the starting node.
            route = ''
            while not routing.IsEnd(index):
                # Convert variable indices to node indices in the displayed route.
                route += str(city_names[routing.IndexToNode(index)]) + ' -> '
                #To display the route in the output, we first get the starting index of the route.
                #index is the index of the variable that represents the starting node of the route, called the depot. 
                #The index of a node variable is not always the same as the index of the node itself (its row number in the locations array)
                #The program iterates through the nodes of the route by:
                index = assignment.Value(routing.NextVar(index))
                #which takes an index for a node variable and returns the index for the next node variable in the route

            #So, when we add nodes to the route displayed in the output, we convert the indices of node variable to node indices, 
            #using the method routing.IndexToNode()
            route += str(city_names[routing.IndexToNode(index)])
            print ("Route:\n\n" + route)
        else:
            print('No solution found.') 
    else:
        print('Specify an instance greater than 0.') 



## 3. Solve Routing Problem

In [13]:
if __name__ == '__main__':
    main()

Total distance: 4989 miles

Route:

New York -> Chicago -> Los Angeles -> Minneapolis -> New York


In the previous example, the distances between cities are explicitly defined in an array. You can also define the distance callback by a function that calculates the distance between any two cities. The next example uses a function that calculates the distance between any two points on Earth from their latitudes and longitudes.

In [8]:
# Distance callback

class CreateDistanceCallback(object):
    """Create callback to calculate distances between points."""

    def __init__(self):

        # Latitudes and longitudes of selected U.S. cities

        locations = [[40.71,  -74.01], # New York
                     [34.05, -118.24], # Los Angeles
                     [41.88,  -87.63], # Chicago
                     [44.98,  -93.27], # Minneapolis
                     [39.74, -104.99], # Denver
                     [32.78,  -96.89], # Dallas
                     [47.61, -122.33], # Seattle
                     [42.36,  -71.06], # Boston
                     [37.77, -122.42], # San Francisco
                     [38.63,  -90.20], # St. Louis
                     [29.76,  -95.37], # Houston
                     [33.45, -112.07], # Phoenix
                     [40.76, -111.89]] # Salt Lake City


        """Create the distance matrix."""
        size = len(locations)
        self.matrix = {}

        for from_node in range(size):
            self.matrix[from_node] = {}
            for to_node in range(size):
                if from_node == to_node:
                    self.matrix[from_node][to_node] = 0
                else:
                    x1 = locations[from_node][0]
                    y1 = locations[from_node][1]
                    x2 = locations[to_node][0]
                    y2 = locations[to_node][1]
                    self.matrix[from_node][to_node] = distance(x1, y1, x2, y2)

    def Distance(self, from_node, to_node):
        return int(self.matrix[from_node][to_node])

In [9]:
def distance(lat1, long1, lat2, long2):
    # Note: The formula used in this function is not exact, as it assumes
    # the Earth is a perfect sphere.

    # Mean radius of Earth in miles
    radius_earth = 3959

    # Convert latitude and longitude to
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
    phi1 = lat1 * degrees_to_radians
    phi2 = lat2 * degrees_to_radians
    lambda1 = long1 * degrees_to_radians
    lambda2 = long2 * degrees_to_radians
    dphi = phi2 - phi1
    dlambda = lambda2 - lambda1

    a = haversine(dphi) + math.cos(phi1) * math.cos(phi2) * haversine(dlambda)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius_earth * c
    return d

def haversine(angle):
    h = math.sin(angle / 2) ** 2
    return h

In [10]:
if __name__ == '__main__':
    main()

Total distance: 4989 miles

Route:

New York -> Chicago -> Los Angeles -> Minneapolis -> New York
